In [ ]:
#@title Download data from GCP bucket
import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

In [ ]:
#@title Imports and moving to working directory
import torch 
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader


# move to data folder
%cd indaba-data

In [ ]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model 
# Tensor shape of embedded data:  [data_len,1280] 
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt")
mut_emb = torch.load("train/train_mut.pt")
df = pd.read_csv("train/train.csv")

In [ ]:
len(df)

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Split data into train and validation
wt_emb_train, wt_emb_val, mut_emb_train, mut_emb_val, df_train, df_val = train_test_split(wt_emb, mut_emb, df, test_size=0.21, random_state=42)

# Define the dataset class
class EmbeddingDataset(Dataset):
  def __init__(self, wt_pt, mut_pt, data_df):
    self.pt_wt = wt_pt
    self.pt_mut = mut_pt
    self.df = data_df

  def __len__(self):
    return len(self.pt_wt)

  def __getitem__(self, index):
    if "ddg" in self.df.columns:
      df_out = torch.Tensor([self.df.iloc[index]["ddg"]])
    else:
      df_out = torch.Tensor([self.df.iloc[index]["ID"]])

    return self.pt_wt[index,:], self.pt_mut[index,:], df_out

# Create separate datasets for the training and validation sets
train_dataset = EmbeddingDataset(wt_emb_train, mut_emb_train, df_train.reset_index(drop=True))
val_dataset = EmbeddingDataset(wt_emb_val, mut_emb_val, df_val.reset_index(drop=True))

# Create dataloaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=27, shuffle=False, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=27, shuffle=False, num_workers=2)


In [ ]:
len(train_dataloader)

In [ ]:
len(val_dataloader)


In [ ]:
for i, (wt_emb, mut_emb, ddg) in enumerate(train_dataloader):
    print(f'wt_emb shape: {wt_emb.shape}, mut_emb shape: {mut_emb.shape}')
    if i > 5:  # Print shapes for first 5 batches only
        break


In [ ]:
for wt_emb, mut_emb, ddg in train_dataloader:
    print(wt_emb_train.shape, mut_emb_train.shape, ddg.shape)
    break
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class DeepFFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepFFNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dp1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp2 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.dp3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, wt_emb, mut_emb):
        x = torch.cat((wt_emb, mut_emb), dim=1)
        
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dp1(x)
        
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dp2(x)
        
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dp3(x)
        
        out = self.fc4(x)
        
        return out

input_dim = 1280 * 2  # Multiply by 2 since we're concatenating wt_emb and mut_emb
hidden_dim = 512
output_dim = 1  # ddG value

model = DeepFFNN(input_dim, hidden_dim, output_dim)


In [ ]:
# Training loop

num_epochs = 20  # Adjust as needed

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    train_losses = []

    for wt_emb, mut_emb, ddg in train_dataloader:
        optimizer.zero_grad()
        outputs = model(wt_emb, mut_emb)
        loss = criterion(outputs, ddg)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    with torch.no_grad():
        val_losses = []
        for wt_emb, mut_emb, ddg in val_dataloader:
            outputs = model(wt_emb, mut_emb)
            loss = criterion(outputs, ddg)
            val_losses.append(loss.item())
    
    avg_val_loss = sum(val_losses) / len(val_losses)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')


Epoch 1, Train Loss: 0.9723331682508415, Val Loss: 0.829824508617784
Epoch 2, Train Loss: 0.8587230365266985, Val Loss: 0.7939834720678867
Epoch 3, Train Loss: 0.8310866027971529, Val Loss: 0.7711610849545032
Epoch 4, Train Loss: 0.8155968670057497, Val Loss: 0.762554890339094
Epoch 5, Train Loss: 0.8042504556913224, Val Loss: 0.7538969334309362
Epoch 6, Train Loss: 0.7948015376062154, Val Loss: 0.7397816039105956
Epoch 7, Train Loss: 0.7860062614463987, Val Loss: 0.7332158229295115
Epoch 8, Train Loss: 0.7792703882219494, Val Loss: 0.727293760373743
Epoch 9, Train Loss: 0.7724689734030648, Val Loss: 0.7206938563142362
Epoch 10, Train Loss: 0.7687712052165923, Val Loss: 0.7144088046459641
Epoch 11, Train Loss: 0.7632086129750052, Val Loss: 0.7094716422550164
Epoch 12, Train Loss: 0.759529313374947, Val Loss: 0.7062453040955331
Epoch 13, Train Loss: 0.7539119569768706, Val Loss: 0.7030017070198799
Epoch 14, Train Loss: 0.7501003997540167, Val Loss: 0.6947250690056075
Epoch 15, Train Los

## Prediction & submission

In [ ]:
# load embedding tensors & traing csv
wt_test_emb = torch.load("test/test_wt.pt")
mut_test_emb = torch.load("test/test_mut.pt")
df_test = pd.read_csv("test/test.csv")

In [ ]:
# creating testing dataset and loading the embedding
test_dataset = EmbeddingDataset(wt_test_emb,mut_test_emb,df_test)
# preparing a dataloader for the testing
test_dataloader = torch.utils.data.dataloader.DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=2,
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
df_result = pd.DataFrame()
with torch.no_grad():
  for batch_idx, (data_mut,data_wt , target) in tqdm(enumerate(test_dataloader)):
    x1 = data_wt.to(device)
    x2 = data_mut.to(device)
    id = target.to(device)
    # make prediction
    y_pred = model(x1,x2)
    df_result = pd.concat([df_result, pd.DataFrame({"ID":id.squeeze().cpu().numpy().astype(int) , "ddg" : y_pred.squeeze().cpu().numpy()})])

In [ ]:
df_result.to_csv("submission_deepffnn_BS26.csv",index=False)